# LogicTune: Fine-Tuning Language Models Using Formal Methods Feedback

This notebook demonstrates the complete pipeline for training language models with automated verification-based feedback.

**Paper Reference:** "Fine-Tuning Language Models Using Formal Methods Feedback"

**Key Techniques:**
- Direct Preference Optimization (DPO)
- Formal verification for automated feedback
- LoRA for efficient fine-tuning


In [ ]:
import sys
sys.path.insert(0, 'src')

from logictune import (
    build_traffic_intersection_model,
    parse_response_to_fsa,
    score_response,
    DPODatasetGenerator,
    train_dpo,
    test_trained_model,
    compare_models,
    evaluate_model_simple
)


KeyboardInterrupt: 

## Step 1: Build Environment Model

Create a formal model of the traffic intersection environment.


In [ ]:
# Build the transition system
system = build_traffic_intersection_model()

print(f"System states: {len(system.get_all_states())}")
print(f"System transitions: {len(system.get_all_transitions())}")
print(f"Atomic propositions: {system.atomic_propositions}")


## Step 2: Test Formal Verification

Demonstrate how the verification pipeline scores controller responses.


In [ ]:
# Safe controller response
safe_response = """
1. If the light is green, go straight through the intersection.
2. If the light is yellow, slow down and stop.
3. If the light is red, stop and wait.
"""

print("Testing SAFE controller:")
print(safe_response)

# Parse and score
safe_fsa = parse_response_to_fsa(safe_response, verbose=True)
safe_score, safe_results = score_response(system, safe_fsa, verbose=True)

print(f"\nSafe Controller Score: {safe_score}/3")


In [ ]:
# Unsafe controller response
unsafe_response = """
1. Always go straight regardless of the light color.
2. Speed through yellow lights.
3. Turn left on red lights.
"""

print("Testing UNSAFE controller:")
print(unsafe_response)

# Parse and score
unsafe_fsa = parse_response_to_fsa(unsafe_response, verbose=True)
unsafe_score, unsafe_results = score_response(system, unsafe_fsa, verbose=True)

print(f"\nUnsafe Controller Score: {unsafe_score}/3")
print(f"\nScore difference: {safe_score - unsafe_score} (used for DPO training)")


## Step 3: Generate DPO Training Dataset

Generate preference pairs using formal verification as automated feedback.

**Note:** This step requires GPU and downloads model weights (~4GB for TinyLlama).


In [ ]:
# Initialize dataset generator
generator = DPODatasetGenerator(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device="auto"
)

# Generate dataset
generator.generate_dataset(
    output_path="dpo_dataset.jsonl",
    n_responses_per_prompt=4,
    temperature=1.0,
    max_pairs_per_prompt=3
)


## Step 4: Train with DPO

Fine-tune the model using Direct Preference Optimization with the generated dataset.

**Note:** This requires GPU for efficient training.


In [ ]:
# Train model
train_dpo(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset_path="dpo_dataset.jsonl",
    output_dir="dpo_model",
    num_epochs=3,
    batch_size=2,
    learning_rate=5e-5,
    beta=0.1,
    use_quantization=False
)


## Step 5: Test Trained Model

Generate responses from the trained model and verify them.


In [ ]:
# Test prompt
test_prompt = "Generate a step-by-step controller for safely navigating a traffic intersection with lights."

# Generate response from trained model
response = test_trained_model("dpo_model", test_prompt)

print(f"Prompt: {test_prompt}\n")
print(f"Generated Response:\n{response}\n")


In [ ]:
# Verify the generated response
print("Verifying generated response...\n")

try:
    controller_fsa = parse_response_to_fsa(response, verbose=True)
    score, results = score_response(system, controller_fsa, verbose=True)
    print(f"\n✓ Verification Score: {score}/3")
except Exception as e:
    print(f"✗ Error during verification: {e}")


## Step 6: Evaluate Model Improvement

Compare base model vs fine-tuned model to measure improvement in specification satisfaction rate.


In [ ]:
# Compare base model vs fine-tuned model
comparison = compare_models(
    base_model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    fine_tuned_model="dpo_model",
    test_prompts=None,  # Uses default test prompts
    n_samples_per_prompt=2,  # 2 samples per prompt for faster evaluation
    output_file="evaluation_results.json"
)

print(f"\n{'='*70}")
print(f"Base Model: {comparison['base_model']['satisfaction_rate']:.1f}%")
print(f"Fine-Tuned Model: {comparison['fine_tuned_model']['satisfaction_rate']:.1f}%")
print(f"Improvement: {comparison['improvement']:+.1f}%")
print(f"{'='*70}")


## Summary

This notebook demonstrated the complete LogicTune pipeline:

1. **Environment Model**: Built a formal transition system for traffic intersection
2. **Verification**: Scored controller responses using formal methods
3. **Dataset Generation**: Created preference pairs with automated feedback
4. **DPO Training**: Fine-tuned model to prefer safe controllers
5. **Evaluation**: Tested and verified the trained model
6. **Comparison**: Measured improvement in specification satisfaction rate

The key innovation is using formal verification instead of human feedback for preference learning, enabling automated and reliable fine-tuning for safety-critical control tasks.


## Summary

This notebook demonstrated the complete LogicTune pipeline:

1. **Environment Model**: Built a formal transition system for traffic intersection
2. **Verification**: Scored controller responses using formal methods
3. **Dataset Generation**: Created preference pairs with automated feedback
4. **DPO Training**: Fine-tuned model to prefer safe controllers
5. **Evaluation**: Tested and verified the trained model

The key innovation is using formal verification instead of human feedback for preference learning.
